<h1 align="center"><strong>Relazione di Open Data Management 2023-2024</strong></h1>
<h3 align="center"><strong><em>di Daniele Nicosia e Claudio Bellanti</em></strong></h3>
<h5 align="center"><em>Università degli Studi di Palermo - Facoltà di Informatica</em></h5>

***
***

### [`1. - Traccia`](#1-traccia)
### [`2. - Selezione dei dataset`](#2-selezione-dei-dataset)
- ##### [`2.1 - Raccolta`](#21-raccolta)
- ##### [`2.2 - Licenze`](#22-licenze)
### [`3. - Elaborazione dei dataset`](#3-elaborazione-dei-dataset)
- ##### [`3.1 - Pulizia e selezione dei dati rilevanti`](#31-pulizia-e-selezione-dei-dati-rilevanti)
- ##### [`3.2 - Arricchimento`](#32-arricchimento)
### [`4. - Finalizzazione dei dataset`](#4-finalizzazione-dei-dataset)
- ##### [`4.1 - Ontologia`](#41-ontologia)
- ##### [`4.2 - Conversione in RDF`](#42-conversione-in-rdf)
### [`5. - Visualizzazione dati`](#5-visualizzazione-dati)

***
***

## **`1. - Traccia`**

Utilizzando il linguaggio di programmazione Python, per lo sviluppo del progetto si devono innanzitutto rispettare i seguenti passi:

- _Selezione dati_
- _Elaborazione dati (data cleaning, definizione struttura omogenea)_
- _Open Linked Data (creazione di uno strato semantico, ontologie, interlinking)_

L'obiettivo della suddetta relazione è lo sviluppo di ???

***
***

## **`2. - Selezione dei dataset`**

#### **`2.1 - Raccolta`**

Per lo sviluppo del progetto sono stati selezionati dei dataset i cui dati rappresentino gli eventi che si sono verificati nel Comune di Milano nel periodo 2014-2021. I dataset selezionati sono i seguenti:

- [**`Parco veicoli circolanti per classe di inquinamento 2007-2022`**](https://dati.comune.milano.it/dataset/ds530-emissioni-inquinanti-standard-eu-2007-avanti)
- [**`Ricoveri ordinari apparato respiratorio 2007-2021`**](https://dati.comune.milano.it/dataset/ds1053_ricoveri-ordinari-apparato-respiratorio)
- [**`Stazioni di monitoraggio inquinanti`**](https://dati.comune.milano.it/dataset/ds484_stazioni_di_monitoraggio_inquinanti_atmosferici_dellarpa_sit)
- [**`Ricostruzione della popolazione 2002-2019`**](https://demo.istat.it/app/?i=RIC&l=it)
  - Per questo dataset è stato necessario effettuare una personalizzazione tramite l'interfaccia di ISTAT. Il dataset risultante contiene la popolazione residente nel solo Comune di Milano dal 2002 al 2019.
- [**`Popolazione residente 2019-2023`**](https://demo.istat.it/app/?i=POS&l=it)
  - Per questo dataset è stato necessario effettuare una personalizzazione tramite l'interfaccia di ISTAT. Il dataset risultante contiene la popolazione residente nel solo Comune di Milano dal 2020 al 2022.
- [**`Standard di emissioni CO europei`**](https://en.wikipedia.org/wiki/European_emission_standards#Toxic_emission:_stages_and_legal_framework)
  - Per questo dataset è stato necessario effettuare uno scraping del sito. Il dataset risultante contiene gli standard di emissioni CO europei dal 1992 al 2021.

***

#### **`2.2 - Licenze`**

Le licenze dei dataset selezionati sono le seguenti:

- **`Parco veicoli circolanti per classe di inquinamento 2007-2022`**: Creative Commons Attribuzione 4.0 Internazionale ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/))
- **`Ricoveri ordinari apparato respiratorio 2007-2021`**: Creative Commons Attribuzione-Condividi allo stesso modo 3.0 Italia ([CC BY-SA 3.0 IT](https://creativecommons.org/licenses/by-sa/3.0/it/))
- **`Stazioni di monitoraggio inquinanti`**: Creative Commons Attribuzione 4.0 Internazionale ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/))
- **`Ricostruzione della popolazione 2002-2019`**: Creative Commons Attribuzione 3.0 ([CC BY 3.0](https://www.istat.it/it/note-legali))
- **`Popolazione residente 2019-2023`**: Creative Commons Attribuzione 3.0 ([CC BY 3.0](https://creativecommons.org/licenses/by/4.0/))
- **`Standard di emissioni CO europei`**: Creative Commons Attribuzione-ShareAlike 4.0 Internazionale [CC BY-SA 4.0](https://en.wikipedia.org/wiki/Wikipedia:Text_of_the_Creative_Commons_Attribution-ShareAlike_4.0_International_License)

***
***

## **`3. - Elaborazione dei dataset`**

#### **`3.1 - Pulizia e selezione dei dati rilevanti`**

Il primo passo effettuato è stato unire i diversi dataset della popolazione residente nel Comune di Milano, in un unico dataset.

Si è notato che i dataset puri non erano tutti adattati allo stesso modo, infatti, si sono riscontrate due problematiche:

- Encoding dei dataset differenti, a causa delle loro diversi origini
- Errori vari dovuti a `;` o `,` non correttamente inseriti

Inoltre, per il dataset delle emissioni è stato necessario effettuare uno scraping poichè non era disponibile in formato CSV. Per effettuare lo scraping è stato utilizzato il linguaggio di programmazione Python e la libreria `BeautifulSoup`. Lo script utilizzato è accessibile qui: [**`scraping.ipynb`**](dataset/emissioni/scraping.ipynb).

***

#### **`3.2 - Arricchimento`**

Successivamente alla fase di pulizia e selezione, si è introdotta una fase di arricchimento dei dataset.

***
***

## **`4. - Finalizzazione dei dataset`**

#### **`4.1 - Ontologia`**

???

***

#### **`4.2 - Conversione in RDF`**

???

***
***

## **`5. - Visualizzazione dati`**

???

***